# Ragas Evaluation with Llama Stack - Demo [remote execution]

This notebook demonstrates how to use the Ragas out-of-tree provider.


## Run your Llama Stack distribution

Nuke any old distro config files you might have lying around (I find these get in the way whenever I change my `.env` variables): 
```bash
ls ~/.llama/distributions/
rm -r ~/.llama/distributions/<name-of-your-distro>
```

Then, run your llama stack server with:
```bash
dotenv run uv run llama stack run distribution/run.yaml
```




## Setup and Imports


In [1]:
# Install dev packages if not already installed
# !uv pip install -e ".[dev]"

import os
from datetime import datetime

import pandas as pd
from llama_stack_client import LlamaStackClient
from rich.pretty import pprint

from llama_stack_provider_ragas.constants import PROVIDER_ID_INLINE, PROVIDER_ID_REMOTE

/Users/dmanilof/src/llama-stack-provider-ragas/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Llama Stack Client Setup

- Make sure we have an inference model (model_type='llm')
- Make sure we have an embedding model (model_type='embedding')


In [2]:
# If using the remote provider, you will need ngrok to enable remote access to your Llama Stack server
# Otherwise, the base_url is just http://localhost:8321
client = LlamaStackClient(base_url=os.getenv("KUBEFLOW_LLAMA_STACK_URL"))
available_models = client.models.list()
pprint(available_models)

INFO:httpx:HTTP Request: GET https://24700eefcd22.ngrok-free.app/v1/models "HTTP/1.1 200 OK"


[
│   Model(
│   │   id='granite-openshift/granite-3-3-8b-instruct',
│   │   created=1769023016,
│   │   owned_by='llama_stack',
│   │   custom_metadata={
│   │   │   'model_type': 'llm',
│   │   │   'provider_id': 'granite-openshift',
│   │   │   'provider_resource_id': 'granite-3-3-8b-instruct'
│   │   },
│   │   object='model'
│   ),
│   Model(
│   │   id='llama-openshift/llama-3-2-3b-instruct',
│   │   created=1769023016,
│   │   owned_by='llama_stack',
│   │   custom_metadata={
│   │   │   'model_type': 'llm',
│   │   │   'provider_id': 'llama-openshift',
│   │   │   'provider_resource_id': 'llama-3-2-3b-instruct'
│   │   },
│   │   object='model'
│   ),
│   Model(
│   │   id='ollama/all-minilm:latest',
│   │   created=1769023016,
│   │   owned_by='llama_stack',
│   │   custom_metadata={
│   │   │   'model_type': 'embedding',
│   │   │   'provider_id': 'ollama',
│   │   │   'provider_resource_id': 'all-minilm:latest',
│   │   │   'embedding_dimension': 384
│   │   },
│   │   object='model'
│   ),
│   Model(
│   │   id='ollama/granite3.3:2b',
│   │   created=1769023016,
│   │   owned_by='llama_stack',
│   │   custom_metadata={'model_type': 'llm', 'provider_id': 'ollama', 'provider_resource_id': 'granite3.3:2b'},
│   │   object='model'
│   ),
│   Model(
│   │   id='ollama/nomic-embed-text:latest',
│   │   created=1769023016,
│   │   owned_by='llama_stack',
│   │   custom_metadata={
│   │   │   'model_type': 'embedding',
│   │   │   'provider_id': 'ollama',
│   │   │   'provider_resource_id': 'nomic-embed-text:latest',
│   │   │   'embedding_dimension': 768,
│   │   │   'context_length': 8192
│   │   },
│   │   object='model'
│   ),
│   Model(
│   │   id='ollama/all-minilm:l6-v2',
│   │   created=1769023016,
│   │   owned_by='llama_stack',
│   │   custom_metadata={
│   │   │   'model_type': 'embedding',
│   │   │   'provider_id': 'ollama',
│   │   │   'provider_resource_id': 'all-minilm:l6-v2',
│   │   │   'embedding_dimension': 384,
│   │   │   'context_length': 512
│   │   },
│   │   object='model'
│   )
]

## Dataset Preparation

Create a sample RAG evaluation dataset. In a real scenario, you would load your own dataset.


In [3]:
# Sample Ragas evaluation dataset
evaluation_data = [
    {
        "user_input": "What is the capital of France?",
        "response": "The capital of France is Paris.",
        "retrieved_contexts": [
            "Paris is the capital and most populous city of France."
        ],
        "reference": "Paris",
    },
    {
        "user_input": "Who invented the telephone?",
        "response": "Alexander Graham Bell invented the telephone in 1876.",
        "retrieved_contexts": [
            "Alexander Graham Bell was a Scottish-American inventor who patented the first practical telephone."
        ],
        "reference": "Alexander Graham Bell",
    },
    {
        "user_input": "What is photosynthesis?",
        "response": "Photosynthesis is the process by which plants convert sunlight into energy.",
        "retrieved_contexts": [
            "Photosynthesis is a process used by plants to convert light energy into chemical energy."
        ],
        "reference": "Photosynthesis is the process by which plants and other organisms convert light energy into chemical energy.",
    },
]

## Dataset Registration

Register the dataset with Llama Stack's Datasets API using the direct rows approach.


In [4]:
# De-register the dataset if it already exists
dataset_id = "ragas_demo_dataset"
try:
    client.beta.datasets.unregister(dataset_id)
except Exception:
    pass

/var/folders/8g/k40y0tk10_g28ck0h9nxph9m0000gn/T/ipykernel_24442/2867941375.py:4: DeprecationWarning: deprecated
  client.beta.datasets.unregister(dataset_id)
INFO:httpx:HTTP Request: DELETE https://24700eefcd22.ngrok-free.app/v1beta/datasets/ragas_demo_dataset "HTTP/1.1 200 OK"


In [5]:
dataset_response = client.beta.datasets.register(
    dataset_id=dataset_id,
    purpose="eval/question-answer",  # RAG evaluation purpose
    source={"type": "rows", "rows": evaluation_data},
    metadata={
        "provider_id": "localfs",  # seems there's a bug in datasets
        "description": "Sample RAG evaluation dataset for Ragas demo",
        "size": len(evaluation_data),
        "format": "ragas",
        "created_at": datetime.now().isoformat(),
    },
)
pprint(dataset_response)

/var/folders/8g/k40y0tk10_g28ck0h9nxph9m0000gn/T/ipykernel_24442/3550987790.py:1: DeprecationWarning: deprecated
  dataset_response = client.beta.datasets.register(
INFO:httpx:HTTP Request: POST https://24700eefcd22.ngrok-free.app/v1beta/datasets "HTTP/1.1 200 OK"


DatasetRegisterResponse(
│   identifier='ragas_demo_dataset',
│   provider_id='localfs',
│   purpose='eval/question-answer',
│   source=SourceRowsDataSource(
│   │   rows=[
│   │   │   {
│   │   │   │   'user_input': 'What is the capital of France?',
│   │   │   │   'response': 'The capital of France is Paris.',
│   │   │   │   'retrieved_contexts': ['Paris is the capital and most populous city of France.'],
│   │   │   │   'reference': 'Paris'
│   │   │   },
│   │   │   {
│   │   │   │   'user_input': 'Who invented the telephone?',
│   │   │   │   'response': 'Alexander Graham Bell invented the telephone in 1876.',
│   │   │   │   'retrieved_contexts': [
│   │   │   │   │   'Alexander Graham Bell was a Scottish-American inventor who patented the first practical telephone.'
│   │   │   │   ],
│   │   │   │   'reference': 'Alexander Graham Bell'
│   │   │   },
│   │   │   {
│   │   │   │   'user_input': 'What is photosynthesis?',
│   │   │   │   'response': 'Photosynthesis is the process by which plants convert sunlight into energy.',
│   │   │   │   'retrieved_contexts': [
│   │   │   │   │   'Photosynthesis is a process used by plants to convert light energy into chemical energy.'
│   │   │   │   ],
│   │   │   │   'reference': 'Photosynthesis is the process by which plants and other organisms convert light energy into chemical energy.'
│   │   │   }
│   │   ],
│   │   type='rows'
│   ),
│   metadata={
│   │   'provider_id': 'localfs',
│   │   'description': 'Sample RAG evaluation dataset for Ragas demo',
│   │   'size': 3,
│   │   'format': 'ragas',
│   │   'created_at': '2026-01-21T14:16:56.359380'
│   },
│   provider_resource_id='ragas_demo_dataset',
│   type='dataset'
)

## Benchmark Registration

Register a benchmark that defines what metrics to use for evaluation.


In [6]:
# comment out the provider you don't want to run
benchmarks_providers = [
    ("ragas_demo_benchmark__inline", PROVIDER_ID_INLINE),
    ("ragas_demo_benchmark__remote", PROVIDER_ID_REMOTE),
]

for benchmark_id, provider_id in benchmarks_providers:
    benchmark_response = client.alpha.benchmarks.register(
        benchmark_id=benchmark_id,
        dataset_id=dataset_id,
        scoring_functions=[
            "answer_relevancy",  # How relevant is the answer to the question?
            # "context_precision",     # How precise are the retrieved contexts?
            # "faithfulness",          # How faithful is the answer to the contexts?
            # "context_recall",        # How much of the ground truth is covered by contexts?
            # "answer_correctness"  # How correct is the answer compared to ground truth?
        ],
        provider_id=provider_id,
        # metadata={
        #     "provider": "ragas",
        #     "version": "1.0",
        #     "metrics_count": len(ragas_metrics),
        #     "created_at": datetime.now().isoformat()
        # }
    )

pprint(benchmark_response)

/var/folders/8g/k40y0tk10_g28ck0h9nxph9m0000gn/T/ipykernel_24442/1526757944.py:8: DeprecationWarning: deprecated
  benchmark_response = client.alpha.benchmarks.register(
INFO:httpx:HTTP Request: POST https://24700eefcd22.ngrok-free.app/v1alpha/eval/benchmarks "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://24700eefcd22.ngrok-free.app/v1alpha/eval/benchmarks "HTTP/1.1 200 OK"


None

In [7]:
benchmarks = client.alpha.benchmarks.list()
pprint(benchmarks)

INFO:httpx:HTTP Request: GET https://24700eefcd22.ngrok-free.app/v1alpha/eval/benchmarks "HTTP/1.1 200 OK"


[
│   Benchmark(
│   │   dataset_id='ragas_demo_dataset',
│   │   identifier='ragas_demo_benchmark__inline',
│   │   provider_id='trustyai_ragas_inline',
│   │   scoring_functions=['answer_relevancy'],
│   │   metadata={},
│   │   provider_resource_id='ragas_demo_benchmark__inline',
│   │   type='benchmark'
│   ),
│   Benchmark(
│   │   dataset_id='ragas_demo_dataset',
│   │   identifier='ragas_demo_benchmark__remote',
│   │   provider_id='trustyai_ragas_remote',
│   │   scoring_functions=['answer_relevancy'],
│   │   metadata={},
│   │   provider_resource_id='ragas_demo_benchmark__remote',
│   │   type='benchmark'
│   )
]

## Evaluation Execution

Run the evaluation using our Ragas out-of-tree provider.


In [8]:
# Review settings in distributinon/run.yaml, eg., note that
# since we can't set the embedding model in the benchmark config,
# the embedding model is set in the distribution run.yaml file(all-MiniLM-L6-v2)

remote_job = client.alpha.eval.run_eval(
    benchmark_id="ragas_demo_benchmark__remote",
    benchmark_config={
        "eval_candidate": {
            "type": "model",
            "model": "ollama/granite3.3:2b",
            "sampling_params": {"temperature": 0.1, "max_tokens": 100},
        },
        "scoring_params": {},
        # "num_examples": 1,
    },
)
pprint(remote_job)

INFO:httpx:HTTP Request: POST https://24700eefcd22.ngrok-free.app/v1alpha/eval/benchmarks/ragas_demo_benchmark__remote/jobs "HTTP/1.1 200 OK"


Job(
│   job_id='6f8d9ed6-ca9f-451a-ae8e-80d005d368f6',
│   status='in_progress',
│   runtime_config={
│   │   'benchmark_config': {
│   │   │   'eval_candidate': {
│   │   │   │   'type': 'model',
│   │   │   │   'model': 'ollama/granite3.3:2b',
│   │   │   │   'sampling_params': {
│   │   │   │   │   'strategy': {'type': 'greedy'},
│   │   │   │   │   'max_tokens': 100,
│   │   │   │   │   'repetition_penalty': 1.0,
│   │   │   │   │   'stop': None
│   │   │   │   },
│   │   │   │   'system_message': None
│   │   │   },
│   │   │   'scoring_params': {},
│   │   │   'num_examples': None
│   │   },
│   │   'embedding_model': 'ollama/all-minilm:latest',
│   │   'benchmark': {
│   │   │   'identifier': 'ragas_demo_benchmark__remote',
│   │   │   'provider_resource_id': 'ragas_demo_benchmark__remote',
│   │   │   'provider_id': 'trustyai_ragas_remote',
│   │   │   'type': 'benchmark',
│   │   │   'dataset_id': 'ragas_demo_dataset',
│   │   │   'scoring_functions': ['answer_relevancy'],
│   │   │   'metadata': {}
│   │   },
│   │   'ragas_config': {
│   │   │   'batch_size': None,
│   │   │   'show_progress': True,
│   │   │   'raise_exceptions': True,
│   │   │   'experiment_name': None,
│   │   │   'column_map': None
│   │   },
│   │   'kubeflow_config': {
│   │   │   'results_s3_prefix': 's3://public-rhods/ragas-evaluation-pipeline',
│   │   │   's3_credentials_secret_name': 'aws-credentials',
│   │   │   'pipelines_endpoint': 'https://ds-pipeline-ragas-eval-dspa-ragas-eval-pipeline.apps.rosa.diego-ragas.9s6r.p3.openshiftapps.com',
│   │   │   'namespace': 'ragas-eval-pipeline',
│   │   │   'llama_stack_url': 'http://localhost:8321',
│   │   │   'base_image': 'registry.redhat.io/ubi9/python-312:latest@sha256:55e489965ae3bcd954d33a7022c8a4af4774ac2cdfd8a4011f1f28ae3b7b8f8f',
│   │   │   'pipelines_api_token': None
│   │   }
│   },
│   kubeflow_run_id='7a33835f-87fd-42ff-96a2-885f6698712f',
│   result=None
)

In [9]:
# Review settings in distributinon/run.yaml, eg., note that
# since we can't set the embedding model in the benchmark config,
# the embedding model is set in the distribution run.yaml file(all-MiniLM-L6-v2)

inline_job = client.alpha.eval.run_eval(
    benchmark_id="ragas_demo_benchmark__inline",
    benchmark_config={
        "eval_candidate": {
            "type": "model",
            "model": "ollama/granite3.3:2b",
            "sampling_params": {"temperature": 0.1, "max_tokens": 100},
        },
        "scoring_params": {},
        # "num_examples": 1,
    },
)
pprint(inline_job)

INFO:httpx:HTTP Request: POST https://24700eefcd22.ngrok-free.app/v1alpha/eval/benchmarks/ragas_demo_benchmark__inline/jobs "HTTP/1.1 200 OK"


Job(
│   job_id='0',
│   status='in_progress',
│   result=None,
│   eval_config={
│   │   'embedding_model': 'ollama/all-minilm:latest',
│   │   'ragas_config': {
│   │   │   'batch_size': None,
│   │   │   'show_progress': True,
│   │   │   'raise_exceptions': True,
│   │   │   'experiment_name': None,
│   │   │   'column_map': None
│   │   }
│   }
)

## Results Display


In [10]:
# wait a bit for the job to complete
pprint(
    client.alpha.eval.jobs.status(
        benchmark_id="ragas_demo_benchmark__inline", job_id=inline_job.job_id
    )
)

INFO:httpx:HTTP Request: GET https://24700eefcd22.ngrok-free.app/v1alpha/eval/benchmarks/ragas_demo_benchmark__inline/jobs/0 "HTTP/1.1 200 OK"


Job(
│   job_id='0',
│   status='in_progress',
│   result=None,
│   eval_config={
│   │   'embedding_model': 'ollama/all-minilm:latest',
│   │   'ragas_config': {
│   │   │   'batch_size': None,
│   │   │   'show_progress': True,
│   │   │   'raise_exceptions': True,
│   │   │   'experiment_name': None,
│   │   │   'column_map': None
│   │   }
│   }
)

In [11]:
# wait a bit for the job to complete
pprint(
    client.alpha.eval.jobs.status(
        benchmark_id="ragas_demo_benchmark__remote", job_id=remote_job.job_id
    )
)

INFO:httpx:HTTP Request: GET https://24700eefcd22.ngrok-free.app/v1alpha/eval/benchmarks/ragas_demo_benchmark__remote/jobs/6f8d9ed6-ca9f-451a-ae8e-80d005d368f6 "HTTP/1.1 200 OK"


Job(
│   job_id='6f8d9ed6-ca9f-451a-ae8e-80d005d368f6',
│   status='in_progress',
│   runtime_config={
│   │   'benchmark_config': {
│   │   │   'eval_candidate': {
│   │   │   │   'type': 'model',
│   │   │   │   'model': 'ollama/granite3.3:2b',
│   │   │   │   'sampling_params': {
│   │   │   │   │   'strategy': {'type': 'greedy'},
│   │   │   │   │   'max_tokens': 100,
│   │   │   │   │   'repetition_penalty': 1.0,
│   │   │   │   │   'stop': None
│   │   │   │   },
│   │   │   │   'system_message': None
│   │   │   },
│   │   │   'scoring_params': {},
│   │   │   'num_examples': None
│   │   },
│   │   'embedding_model': 'ollama/all-minilm:latest',
│   │   'benchmark': {
│   │   │   'identifier': 'ragas_demo_benchmark__remote',
│   │   │   'provider_resource_id': 'ragas_demo_benchmark__remote',
│   │   │   'provider_id': 'trustyai_ragas_remote',
│   │   │   'type': 'benchmark',
│   │   │   'dataset_id': 'ragas_demo_dataset',
│   │   │   'scoring_functions': ['answer_relevancy'],
│   │   │   'metadata': {}
│   │   },
│   │   'ragas_config': {
│   │   │   'batch_size': None,
│   │   │   'show_progress': True,
│   │   │   'raise_exceptions': True,
│   │   │   'experiment_name': None,
│   │   │   'column_map': None
│   │   },
│   │   'kubeflow_config': {
│   │   │   'results_s3_prefix': 's3://public-rhods/ragas-evaluation-pipeline',
│   │   │   's3_credentials_secret_name': 'aws-credentials',
│   │   │   'pipelines_endpoint': 'https://ds-pipeline-ragas-eval-dspa-ragas-eval-pipeline.apps.rosa.diego-ragas.9s6r.p3.openshiftapps.com',
│   │   │   'namespace': 'ragas-eval-pipeline',
│   │   │   'llama_stack_url': 'http://localhost:8321',
│   │   │   'base_image': 'registry.redhat.io/ubi9/python-312:latest@sha256:55e489965ae3bcd954d33a7022c8a4af4774ac2cdfd8a4011f1f28ae3b7b8f8f',
│   │   │   'pipelines_api_token': None
│   │   }
│   },
│   kubeflow_run_id='7a33835f-87fd-42ff-96a2-885f6698712f',
│   result=None
)

In [12]:
remote_results = client.alpha.eval.jobs.retrieve(
    benchmark_id="ragas_demo_benchmark__remote", job_id=remote_job.job_id
)
pprint(remote_results)

INFO:httpx:HTTP Request: GET https://24700eefcd22.ngrok-free.app/v1alpha/eval/benchmarks/ragas_demo_benchmark__remote/jobs/6f8d9ed6-ca9f-451a-ae8e-80d005d368f6/result "HTTP/1.1 200 OK"


EvaluateResponse(generations=[], scores={})

In [13]:
inline_results = client.alpha.eval.jobs.retrieve(
    benchmark_id="ragas_demo_benchmark__inline", job_id=inline_job.job_id
)
pprint(inline_results)

INFO:httpx:HTTP Request: GET https://24700eefcd22.ngrok-free.app/v1alpha/eval/benchmarks/ragas_demo_benchmark__inline/jobs/0/result "HTTP/1.1 200 OK"


EvaluateResponse(generations=[], scores={})

## Inline vs Remote Side-by-side

In [14]:
pd.DataFrame.from_dict(
    {
        "inline": [
            r["score"] for r in inline_results.scores["answer_relevancy"].score_rows
        ],
        "remote": [
            r["score"] for r in remote_results.scores["answer_relevancy"].score_rows
        ],
    },
).assign(diff=lambda df: df["remote"] - df["inline"])

KeyError: 'answer_relevancy'